In [1]:
import pandas as pd
import geopandas as gpd
import toml
from pathlib import Path
from sqlalchemy import create_engine
from functions import process_network_summary

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1f}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]

# Congestion Levels by County

Congestion levels are defined by the ratio of congested speed to posted speed limits:

- Minimal: Ratio > 0.70
- Moderate: Ratio > 0.50 and <=0.70
- Heavy: > 0.25 and <= 0.50
- Severe: Ratio <= 0.25

AM (7-8)

In [3]:
# # gdf = gpd.read_file(r'../../../inputs/scenario/networks/shapefiles/AM/AM_edges.shp')
# gdf = gpd.read_file(Path(summary_config['sc_run_path'], 'inputs/scenario/networks/shapefiles/AM/AM_edges.shp'))
# gdf.crs = 'EPSG:2285'

# df = process_network_summary()
# # NOTE: for visualization, do not show connectors
# df = df[df['data3'] != 5]
# gdf = gdf.merge(df[df['tod'] == '7to8'][['ij','congestion_index','congestion_category']], left_on='link_id', right_on='ij', how='left')
# gdf['congestion_category'] = gdf['congestion_category'].fillna('Light')
# color_mapping = {'Light': 'grey','Moderate': 'orange','Heavy': 'red','Severe':'black'}
# gdf.plot(color=gdf["congestion_category"].map(color_mapping), alpha=0.3, linewidth=gdf['lanes'], figsize=(20,20)) 

In [4]:
df = process_network_summary()
df = df[df['county']!='Outside Region']

VMT

In [5]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df['Region'] = _df.sum(axis=1)
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

# _df

,King,Kitsap,Pierce,Snohomish,Region
Light,"2,097,083","286,646","997,306","891,347","4,272,382"
Moderate,"556,418","6,894","177,915","150,362","891,590"
Heavy,"506,824","9,536","63,398","87,487","667,244"
Severe,"55,994",686,"4,530","5,665","66,875"
Heavy + Severe,"562,818","10,222","67,928","93,151","734,119"


----
Share of VMT

In [6]:

pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,King,Kitsap,Pierce,Snohomish,Region
Light,55.5%,91.3%,76.1%,72.6%,64.4%
Moderate,14.7%,2.2%,13.6%,12.2%,13.4%
Heavy,13.4%,3.0%,4.8%,7.1%,10.1%
Severe,1.5%,0.2%,0.3%,0.5%,1.0%
Heavy + Severe,14.9%,3.3%,5.2%,7.6%,11.1%


VMT

In [7]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '17to18']
_df = df_am.pivot_table(index='congestion_category',columns='county',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df['Region'] = _df.sum(axis=1)
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

,King,Kitsap,Pierce,Snohomish,Region
Light,"2,451,132","323,032","1,150,470","1,006,763","4,931,397"
Moderate,"600,282","6,264","157,940","167,513","931,999"
Heavy,"360,699","9,445","62,580","65,760","498,484"
Severe,"37,243",832,"4,867","5,332","48,275"
Heavy + Severe,"397,942","10,277","67,448","71,092","546,759"


----
Share of VMT

In [8]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,King,Kitsap,Pierce,Snohomish,Region
Light,63.7%,92.3%,79.7%,76.5%,70.9%
Moderate,15.6%,1.8%,10.9%,12.7%,13.4%
Heavy,9.4%,2.7%,4.3%,5.0%,7.2%
Severe,1.0%,0.2%,0.3%,0.4%,0.7%
Heavy + Severe,10.3%,2.9%,4.7%,5.4%,7.9%


# Congestion by Road Type

VMT

In [9]:
pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '7to8']
_df = df_am.pivot_table(index='congestion_category',columns='facility_type',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,"2,121,363","324,918","1,825,809",292
Moderate,"319,011",NaN,"572,539",39
Heavy,"187,081",NaN,"480,163",0
Severe,"32,878",NaN,"33,997",0
Heavy + Severe,"219,959",0,"514,160",0


----
Share of VMT

In [10]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,73.7%,100.0%,53.3%,88.1%
Moderate,11.1%,NaN,16.7%,11.9%
Heavy,6.5%,NaN,14.0%,0.0%
Severe,1.1%,NaN,1.0%,0.0%
Heavy + Severe,7.6%,0.0%,15.0%,0.0%


VMT

In [11]:

pd.options.display.float_format = '{:0,.0f}'.format
df_am = df[df['tod'] == '17to18']
_df = df_am.pivot_table(index='congestion_category',columns='facility_type',
               aggfunc='sum',values='VMT')
_df = _df.reindex(['Light','Moderate','Heavy','Severe'])
_df.index.name = None
_df.columns.name = None
_df.loc['Heavy + Severe'] = _df.loc[['Heavy','Severe']].sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,"2,441,875","442,668","2,046,454",399
Moderate,"332,125",NaN,"599,874",0
Heavy,"189,672",NaN,"308,807",5
Severe,"32,368",NaN,"15,907",0
Heavy + Severe,"222,040",0,"324,714",5


----
Share of VMT

In [12]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = _df/_df.sum(axis=0)
_df

,Arterial,Connector,Highway,Other
Light,75.9%,100.0%,62.1%,97.3%
Moderate,10.3%,NaN,18.2%,0.0%
Heavy,5.9%,NaN,9.4%,1.3%
Severe,1.0%,NaN,0.5%,0.0%
Heavy + Severe,6.9%,0.0%,9.9%,1.3%


In [13]:
# Congestion in Equity Focus Areas

# Corridor Travel Time

In [14]:
pd.options.display.float_format = '{:0,.1f}'.format

df = pd.read_csv(r'..\..\..\..\outputs\validation\corridor_speeds.csv')

# For each Corridor_Descrption, define as AM or pm beak by finding which direction has max value
df = df[df['tod'].isin(['7to8','17to18']) & (df['Corridor_Number'] <= 20)]

# Corridors are grouped so that directions are [1,2], [3,4]
for i in range(1,21):
# for i in range(7,8):
    _df = df[df['Corridor_Number'] == i]
#     print(_df)
    peak_time = _df.sort_values('auto_time').iloc[-1]['tod']
    df.loc[df['Corridor_Number']==i, 'peak'] = peak_time
    
df.rename(columns={'Corridor_Description': 'Corridor', 'auto_time':'Auto Travel Time (min.)'}, inplace=True)
df[(df['peak'] == '7to8') & (df['tod'] == '7to8')][['Corridor','Auto Travel Time (min.)']]

,Corridor,Auto Travel Time (min.)
15,Everett to Seattle - SB,60.5
24,Tacoma to Seattle - NB,65.2
51,Lynnwood to Bellevue - SB,36.9
60,Tukwila to Bellevue - NB,30.3
78,Auburn to Renton - NB,24.8
96,Redmond to Seattle - WB,28.9
114,Redmond to Bellevue - WB,12.5
132,Issaquah to Bellevue - WB,14.1
159,Bellevue to Seattle via 520 - EB,23.3
168,Bellevue to Seattle via 90 - WB,28.0


In [15]:
df[(df['peak'] == '17to18') & (df['tod'] == '17to18')][['Corridor','Auto Travel Time (min.)']]

,Corridor,Auto Travel Time (min.)
2,Seattle to Everett - NB,52.5
29,Seattle to Tacoma - SB,55.9
38,Bellevue to Lynnwood - NB,31.6
65,Bellevue to Tukwila - SB,27.8
83,Renton to Auburn - SB,21.2
101,Seattle to Redmond - EB,23.4
119,Bellevue to Redmond - EB,10.8
137,Bellevue to Issaquah - EB,12.1
146,Seattle to Bellevue via 520 - WB,17.4
173,Seattle to Bellevue via 90 - EB,24.9


# Delay

In [16]:
df = process_network_summary()
# df = df[df['county']!='Outside Region']

In [17]:
df[['total_delay','county']].groupby('county').sum()[['total_delay']]

,total_delay
county,
King,"149,518.6"
Kitsap,"2,648.5"
Outside Region,2.2
Pierce,"29,815.2"
Snohomish,"29,786.8"
